AMUSE tutorial on high-order bridge
====================

Hierarchical coupling strategies are fundamental parts of AMUSE.
It enables us to combined the output of a wide variety of solvers into a homogeneous solution.
In this example we will be nesting multiple bridges, to show the power of bridge.

In [ ]:
import numpy
from amuse.units import (units, constants)
from amuse.lab import Particles
from amuse.units import nbody_system
from matplotlib import pyplot

In [ ]:
def sun_earth_and_moon():
    stars = Particles(4)
    sun = stars[0]
    sun.mass = units.MSun(1.0)
    sun.position = units.m(numpy.array((0.0,0.0,0.0)))
    sun.velocity = units.ms(numpy.array((0.0,0.0,0.0)))
    sun.radius = units.RSun(1.0)

    earth = stars[1]
    earth.mass = units.kg(5.9736e24)
    earth.radius = units.km(6371) 
    earth.position = units.km(numpy.array((149.5e6,0.0,0.0)))
    earth.velocity = units.ms(numpy.array((0.0,29800,0.0)))

    moon = stars[2]
    moon.mass = units.kg(7.3477e22 )
    moon.radius = units.km(1737.10) 
    moon.position = units.km(numpy.array((earth.x.value_in(units.km) + 384399.0 ,0.0,0.0)))
    moon.velocity = ([0.0,1.022,0] | units.km/units.s) + earth.velocity   
    
    jupiter = stars[3]
    jupiter.mass = 50 | units.MJupiter
    jupiter.position = (1, 0, 0) * (778.e+6 | units.km)
    v = (constants.G*sun.mass/jupiter.x).sqrt()
    jupiter.velocity = (0, 1, 0) * v

    return sun, earth, moon, jupiter

In [ ]:
from amuse.community.ph4.interface import ph4

sun, earth, moon, jupiter = sun_earth_and_moon()
sosy = sun + earth + moon + jupiter

converter=nbody_system.nbody_to_si(sun.mass, earth.position.length())
star_gravity = ph4(converter)
star_gravity.particles.add_particle(sun + jupiter)

planet_gravity = ph4(converter)
planet_gravity.particles.add_particle(earth)

moon_gravity = ph4(converter)
moon_gravity.particles.add_particle(moon)

channel_from_star_to_framework = star_gravity.particles.new_channel_to(sosy)
channel_from_planet_to_framework = planet_gravity.particles.new_channel_to(sosy)
channel_from_moon_to_framework = moon_gravity.particles.new_channel_to(sosy)

In [ ]:
from amuse.couple import bridge
sp_gravity = bridge.Bridge()
sp_gravity.add_system(moon_gravity, (planet_gravity,))
sp_gravity.add_system(planet_gravity, (moon_gravity,))

gravity = bridge.Bridge()
gravity.add_system(sp_gravity, (star_gravity,))
gravity.add_system(star_gravity, (sp_gravity,))

In [ ]:
from amuse.lab import zero
Etot_init = gravity.kinetic_energy + gravity.potential_energy
Etot_prev = Etot_init

sp_gravity.timestep = 1|units.day
gravity.timestep = 10|units.day
time = zero
dt = 20|units.day
t_end = 24| units.yr
r_se= [] | units.au
r_em = [] | units.au
t = [] | units.yr
while time < t_end:
    time += dt
    gravity.evolve_model(time)

    Etot_prev_se = gravity.kinetic_energy + gravity.potential_energy

    channel_from_star_to_framework.copy()
    channel_from_planet_to_framework.copy()
    channel_from_moon_to_framework.copy()
    
    t.append(time)
    r_se.append((sun.position-earth.position).length())
    r_em.append((earth.position-moon.position).length())
             
    Ekin = gravity.kinetic_energy 
    Epot = gravity.potential_energy
    Etot = Ekin + Epot
    print("T=", time.in_(units.yr), end=' ') 
    print("E= ", Etot/Etot_init, "Q= ", Ekin/Epot, end=' ')
    print("dE=", (Etot_init-Etot)/Etot, "ddE=", (Etot_prev-Etot)/Etot) 
    Etot_prev = Etot
gravity.stop()

from amuse.plot import plot
plot(t, r_se)
plot(t, r_em)
pyplot.semilogy()
pyplot.show()